In [5]:
import pandas as pd
import numpy as np

In [6]:
df_sub = pd.read_csv('Rule_based_Confirmation.csv')

In [7]:
def recreate_sub(group):
    group = group[group['score'] != 0]
    if(group.shape[0] == 0):
        return ""
    else:
        group = group.sort_values(by=['score'], ascending=False)
        group.loc[:,'hotel_id'] = group['hotel_id'].apply(str)
        return " ".join(group['hotel_id'])

In [9]:
df_sub = df_sub.groupby('session_id').apply(lambda x: recreate_sub(x)).reset_index(name='item_rule')
df_sub.head()

,session_id,item_rule
0,0000059a39020,
1,0000b9394297b,
2,0001650138d30,
3,000177b850519,17011 17266
4,00017b3b2c136,6698142 6298194


In [23]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [24]:
df_test = pd.read_csv('test_off.csv')
df_test = get_submission_target(df_test)
df_test = (df_test.merge(df_sub, left_on=['session_id'], right_on=['session_id'], how="left"))
df_test.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,item_rule
0,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,3241426 1050068
1,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28,7065316
2,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,3843244
3,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...,2817590
4,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247|559056|1045096|1963879|693596|1967173|...,80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...,1177554 7304020


In [25]:
def complete_predictions(row):               
    l_impressions = row.impressions.split('|')
    if (row.item_rule == ''):
        return " ".join(l_impressions)
    l_recs = row.item_rule.split(' ')
    result = []
    for i in l_recs:
        result.append(str(i))
    for i in l_impressions:
        if(str(i) not in result):
            result.append(str(i))
    return " ".join(result)

In [26]:
df_test['item_recommendations'] = df_test.apply(lambda x : complete_predictions(x), axis=1)

In [27]:
df_test = df_test[['user_id', 'session_id', 'timestamp', 'step', 'item_recommendations']]

In [29]:
df_test.to_csv('rule_based_FR_reconstruct.csv')